In [278]:
import pandas as pd;
import numpy as np;

In [279]:
# Load dataset (replace with your actual data loading method)
survey_df = pd.read_csv("../../data/final_survey_table.csv", low_memory=False)

In [280]:
survey_df

,What is your age (# years)?,What is your gender?,In which country do you currently reside?,What is the highest level of formal education that you have attained or plan to attain within the next 2 years?,Select the title most similar to your current role (or most recent title if retired),What is the size of the company where you are employed?,Approximately how many individuals are responsible for data science workloads at your place of business?,Does your current employer incorporate machine learning methods into their business?,What is your current yearly compensation (approximate $USD)?,Approximately how much money have you spent on machine learning and/or cloud computing products at your work in the past 5 years?,What is the primary tool that you use at work or school to analyze data?,How long have you been writing code to analyze data (at work or at school)?,What programming language would you recommend an aspiring data scientist to learn first?,Have you ever used a TPU (tensor processing unit)?,For how many years have you used machine learning methods?,income_category
0,40-44,Male,Australia,Master’s degree,Other,Enterprise,20+,I do not know,"250,000-299,999","$10,000-$99,999",Local development environments,1-2 years,Python,Used TPU,2-3 years,T20
1,22-24,Male,India,Bachelor’s degree,Other,Small,0,No (we do not use ML methods),"4,000-4,999",$0 (USD),Local development environments,< 1 years,Python,Never,< 1 years,B40
2,22-24,Male,India,Master’s degree,Data Scientist,Medium,20+,We are exploring ML methods (and may one day p...,"10,000-14,999",$100-$999,Local development environments,3-5 years,Python,Used TPU,2-3 years,M40
3,22-24,Female,United States of America,Bachelor’s degree,Data Scientist,Enterprise,20+,"We recently started using ML methods (i.e., mo...","80,000-89,999",$0 (USD),Local development environments,3-5 years,Python,Used TPU,3-4 years,T20
4,55-59,Male,Netherlands,Master’s degree,Other,Small,1-2,We are exploring ML methods (and may one day p...,$0-999,$100-$999,Local development environments,5-10 years,Python,Never,< 1 years,B40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,25-29,Male,India,Master’s degree,Data Scientist,Small,1-2,"We recently started using ML methods (i.e., mo...","1,000-1,999",$100-$999,Local development environments,3-5 years,Python,Never,2-3 years,B40
7795,22-24,Female,Other,Bachelor’s degree,Other,Medium,1-2,We are exploring ML methods (and may one day p...,"5,000-7,499",$100-$999,Local development environments,1-2 years,Python,Never,1-2 years,M40
7796,25-29,Male,China,I prefer not to answer,Data Engineer,Mid-Large,5-9,"We recently started using ML methods (i.e., mo...","20,000-24,999",$100-$999,Local development environments,1-2 years,Python,Used TPU,1-2 years,M40
7797,25-29,Male,Australia,Bachelor’s degree,Other,Large,5-9,No (we do not use ML methods),"60,000-69,999","$10,000-$99,999",Local development environments,3-5 years,Python,Never,1-2 years,T20


In [281]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Data columns (total 16 columns):
 #   Column                                                                                                                             Non-Null Count  Dtype 
---  ------                                                                                                                             --------------  ----- 
 0   What is your age (# years)?                                                                                                        7799 non-null   object
 1   What is your gender?                                                                                                               7799 non-null   object
 2   In which country do you currently reside?                                                                                          7799 non-null   object
 3   What is the highest level of formal education that you have attained or plan to attain within

In [282]:
# Define the column mapping
column_mapping = {
    "What is your age (# years)?": "age",
    "What is your gender?": "gender",
    "In which country do you currently reside?": "country",
    "What is the highest level of formal education that you have attained or plan to attain within the next 2 years?": "education_level",
    "Select the title most similar to your current role (or most recent title if retired)": "job_title",
    "What is the size of the company where you are employed?": "company_size",
    "Approximately how many individuals are responsible for data science workloads at your place of business?": "data_science_team_size",
    "Does your current employer incorporate machine learning methods into their business?": "ml_incorporation",
    "What is your current yearly compensation (approximate $USD)?": "yearly_compensation",
    "Approximately how much money have you spent on machine learning and/or cloud computing products at your work in the past 5 years?": "ml_spending",
    "What is the primary tool that you use at work or school to analyze data?": "primary_tool",
    "How long have you been writing code to analyze data (at work or at school)?": "coding_experience",
    "What programming language would you recommend an aspiring data scientist to learn first?": "recommended_language",
    "Have you ever used a TPU (tensor processing unit)?": "used_tpu",
    "For how many years have you used machine learning methods?": "ml_experience",
    "income_category": "income_category"
}

# Rename the columns
survey_df.rename(columns=column_mapping, inplace=True)

In [283]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   age                     7799 non-null   object
 1   gender                  7799 non-null   object
 2   country                 7799 non-null   object
 3   education_level         7799 non-null   object
 4   job_title               7799 non-null   object
 5   company_size            7799 non-null   object
 6   data_science_team_size  7799 non-null   object
 7   ml_incorporation        7799 non-null   object
 8   yearly_compensation     7799 non-null   object
 9   ml_spending             7799 non-null   object
 10  primary_tool            7799 non-null   object
 11  coding_experience       7799 non-null   object
 12  recommended_language    7799 non-null   object
 13  used_tpu                7799 non-null   object
 14  ml_experience           7799 non-null   object
 15  inco

### 4.3.2.1 Domain-Specific Features

In [284]:
# Define a helper function to convert ranges to midpoints for experience
def convert_years_to_midpoint(range_str):
    if isinstance(range_str, str) and '-' in range_str:
        # Handle ranges like "2-3 years"
        low, high = map(int, range_str.replace(' years', '').split('-'))
        return (low + high) / 2
    elif isinstance(range_str, str) and range_str.startswith('<'):
        # Handle "less than" cases like "< 1 years"
        return 0.5  # Assign midpoint for "< 1 years"
    elif isinstance(range_str, str) and range_str.startswith('>') or range_str.endswith('+ years'):
        # Handle "greater than" cases like "> 10 years" or "20+ years"
        range_str = range_str.replace('>', '').replace('+ years', '').strip()
        return float(range_str) + 1  # Assume 1 year beyond the given value
    elif isinstance(range_str, str):
        # Handle direct numeric values
        return float(range_str.replace(' years', '').strip())
    return np.nan

# Apply conversion
survey_df['ML_Experience_Midpoint'] = survey_df[
    'ml_experience'
].apply(convert_years_to_midpoint).round().astype(int)

survey_df['Coding_Experience_Midpoint'] = survey_df[
    'coding_experience'
].apply(convert_years_to_midpoint).round().astype(int)

# Calculate Interaction Feature: ML Experience to Coding Ratio
survey_df['Total_Experience'] = (
    survey_df['ML_Experience_Midpoint'] +
    survey_df['Coding_Experience_Midpoint']
).round().astype(int)


In [285]:
# Drop the unused features
survey_df.drop(columns=[
                        'ML_Experience_Midpoint', 'Coding_Experience_Midpoint',
                        'ml_experience', 'coding_experience'
                        ], inplace=True)


In [286]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   age                     7799 non-null   object
 1   gender                  7799 non-null   object
 2   country                 7799 non-null   object
 3   education_level         7799 non-null   object
 4   job_title               7799 non-null   object
 5   company_size            7799 non-null   object
 6   data_science_team_size  7799 non-null   object
 7   ml_incorporation        7799 non-null   object
 8   yearly_compensation     7799 non-null   object
 9   ml_spending             7799 non-null   object
 10  primary_tool            7799 non-null   object
 11  recommended_language    7799 non-null   object
 12  used_tpu                7799 non-null   object
 13  income_category         7799 non-null   object
 14  Total_Experience        7799 non-null   int64 
dtypes: i

## Data Encoding

In [287]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Function to dynamically map ordinal categories
def dynamic_ordinal_mapping(df, columns):
    """
    Dynamically maps ordinal values for specified columns in a dataframe.
    Automatically assigns integer values based on the order of unique categories.

    Args:
        df (pd.DataFrame): Input dataframe.
        columns (list): List of columns to apply ordinal mapping.

    Returns:
        pd.DataFrame: Dataframe with ordinal-encoded columns.
        dict: Dictionary with column names as keys and their value-to-ordinal mappings as values.
    """
    column_mappings = {}  # To store the mappings for each column

    for col in columns:
        # Extract unique categories and sort them logically
        unique_categories = sorted(df[col].unique())
        ordinal_mapping = {category: i for i, category in enumerate(unique_categories)}
        
        # Store the mapping for the current column
        column_mappings[col] = ordinal_mapping

        # Replace categories with ordinal values
        df[col] = df[col].map(ordinal_mapping)
        
    return column_mappings, df

# Apply to ordinal columns in your dataset
ordinal_columns = [
    "age",
    "data_science_team_size",
    "yearly_compensation",
    "education_level",
    "ml_spending"
]

ordinal_map, survey_df = dynamic_ordinal_mapping(survey_df, ordinal_columns)

# Step 3: One-Hot Encoding for Nominal Categorical Variables
# Justification: Nominal data has no inherent order (e.g., gender, country).

nominal_columns = [
    "gender",
    "country",
    "primary_tool",
    "job_title",
    "ml_incorporation",
    "primary_tool",
    "recommended_language",
    "income_category"
]

survey_df = pd.get_dummies(survey_df, columns=nominal_columns, drop_first=True)

# Step 4: Label Encoding for Binary Columns
# Justification: Label encoding is efficient for binary columns.

binary_columns = ["used_tpu"]

for col in binary_columns:
    le = LabelEncoder()
    survey_df[col] = le.fit_transform(survey_df[col])

# Final Memory Optimization: Convert category columns back to 'category' dtype
for col in survey_df.select_dtypes(include='category').columns:
    survey_df[col] = survey_df[col].astype('category')

# Display the final optimized dataframe
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Data columns (total 96 columns):
 #   Column                                                                                                      Non-Null Count  Dtype 
---  ------                                                                                                      --------------  ----- 
 0   age                                                                                                         7799 non-null   int64 
 1   education_level                                                                                             7799 non-null   int64 
 2   company_size                                                                                                7799 non-null   object
 3   data_science_team_size                                                                                      7799 non-null   int64 
 4   yearly_compensation                                                     

## Feature Engineering

### 4.3.2.2 Automated Feature Generation

In [288]:
fact_table = survey_df[['age', 'gender_Male', 'education_level', 
                        'yearly_compensation', 'ml_spending', 'used_tpu', 
                        'Total_Experience']]

# Company Details Dimension
company_details_table = survey_df[['company_size', 'data_science_team_size']]

dim_country = survey_df[[col for col in survey_df.columns if col.startswith('country_')]]

dim_job_title = survey_df[[col for col in survey_df.columns if col.startswith('job_title_')]]

dim_ml_incorporation = survey_df[[col for col in survey_df.columns if col.startswith('ml_incorporation_')]]

dim_primary_tool = survey_df[[col for col in survey_df.columns if col.startswith('primary_tool_')]]

dim_recommended_language = survey_df[[col for col in survey_df.columns if col.startswith('recommended_language_')]]

dim_income_category = survey_df[[col for col in survey_df.columns if col.startswith('income_category_')]]

## Dimensionality Reduction

### 4.3.3.1 Remove Features with Poor Generalization
1. Drop or ignore dim_income_category as the income category B40, M40, T20 is only in Malaysia and it event didnt have same standard accross state in Malaysia.

In [289]:
dim_income_category

,income_category_M40,income_category_T20
0,False,True
1,False,False
2,True,False
3,False,True
4,False,False
...,...,...
7794,False,False
7795,True,False
7796,True,False
7797,False,True


2. Remove duplicate column in dim_primary_tool

In [290]:
dim_primary_tool.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Data columns (total 20 columns):
 #   Column                                         Non-Null Count  Dtype
---  ------                                         --------------  -----
 0   primary_tool_Basic statistical software        7799 non-null   bool 
 1   primary_tool_Basic statistical software        7799 non-null   bool 
 2   primary_tool_Business intelligence software    7799 non-null   bool 
 3   primary_tool_Business intelligence software    7799 non-null   bool 
 4   primary_tool_Cloud-based data software & APIs  7799 non-null   bool 
 5   primary_tool_Cloud-based data software & APIs  7799 non-null   bool 
 6   primary_tool_Local development environments    7799 non-null   bool 
 7   primary_tool_Local development environments    7799 non-null   bool 
 8   primary_tool_Other                             7799 non-null   bool 
 9   primary_tool_Other                             7799 non-null   bool 
 10  

In [291]:
dim_primary_tool = dim_primary_tool.loc[:, ~dim_primary_tool.columns.duplicated()]
dim_primary_tool

,primary_tool_Basic statistical software,primary_tool_Business intelligence software,primary_tool_Cloud-based data software & APIs,primary_tool_Local development environments,primary_tool_Other
0,False,False,False,True,False
1,False,False,False,True,False
2,False,False,False,True,False
3,False,False,False,True,False
4,False,False,False,True,False
...,...,...,...,...,...
7794,False,False,False,True,False
7795,False,False,False,True,False
7796,False,False,False,True,False
7797,False,False,False,True,False


3. Map dim_ml_incorporation to oridinal value rank experience from low to high

In [292]:
# Define the ordinal mapping based on experience (low to high)
ordinal_mapping = {
    'ml_incorporation_No (we do not use ML methods)': 0,
    'ml_incorporation_We use ML methods for generating insights (but do not put working models into production)': 1,
    'ml_incorporation_We are exploring ML methods (and may one day put a model into production)': 2,
    'ml_incorporation_We recently started using ML methods (i.e., models in production for less than 2 years)': 3,
    'ml_incorporation_We have well established ML methods (i.e., models in production for more than 2 years)': 4,
}
ordinal_map["ml_experience"] = {key.replace('ml_incorporation_', ''): value for key, value in ordinal_mapping.items()}

# Apply the lambda function and assign the result to a new column
dim_ml_incorporation = dim_ml_incorporation.copy()
dim_ml_incorporation.loc[:, 'ml_experience'] = dim_ml_incorporation.apply(
    lambda x: max([ordinal_mapping[col] for col in dim_ml_incorporation.columns if x[col]] or [0]), axis=1
)

# Drop unused feature
dim_ml_incorporation = dim_ml_incorporation.drop(columns=ordinal_mapping.keys())
dim_ml_incorporation

,ml_experience
0,0
1,0
2,2
3,3
4,2
...,...
7794,3
7795,2
7796,3
7797,0


In [293]:
# Concat dim_ml_incorporation to fact_table
fact_table = pd.concat([fact_table, dim_ml_incorporation], axis=1)
fact_table

,age,gender_Male,education_level,yearly_compensation,ml_spending,used_tpu,Total_Experience,ml_experience
0,5,True,3,11,2,1,4,0
1,1,True,0,15,0,0,0,0
2,1,True,3,2,3,1,6,2
3,1,False,0,22,0,1,8,3
4,8,True,3,0,3,0,8,2
...,...,...,...,...,...,...,...,...
7794,2,True,3,1,3,0,6,3
7795,1,False,0,17,3,0,4,2
7796,2,True,2,8,3,1,4,3
7797,2,True,0,19,2,0,6,0


In [294]:
# Save the mapping between columns value to ordinal value
ordinal_map

{'age': {'18-21': 0,
  '22-24': 1,
  '25-29': 2,
  '30-34': 3,
  '35-39': 4,
  '40-44': 5,
  '45-49': 6,
  '50-54': 7,
  '55-59': 8,
  '60-69': 9,
  '70+': 10},
 'data_science_team_size': {'0': 0,
  '1-2': 1,
  '10-14': 2,
  '15-19': 3,
  '20+': 4,
  '3-4': 5,
  '5-9': 6},
 'yearly_compensation': {'$0-999': 0,
  '1,000-1,999': 1,
  '10,000-14,999': 2,
  '100,000-124,999': 3,
  '125,000-149,999': 4,
  '15,000-19,999': 5,
  '150,000-199,999': 6,
  '2,000-2,999': 7,
  '20,000-24,999': 8,
  '200,000-249,999': 9,
  '25,000-29,999': 10,
  '250,000-299,999': 11,
  '3,000-3,999': 12,
  '30,000-39,999': 13,
  '300,000-500,000': 14,
  '4,000-4,999': 15,
  '40,000-49,999': 16,
  '5,000-7,499': 17,
  '50,000-59,999': 18,
  '60,000-69,999': 19,
  '7,500-9,999': 20,
  '70,000-79,999': 21,
  '80,000-89,999': 22,
  '90,000-99,999': 23,
  '> $500,000': 24},
 'education_level': {'Bachelor’s degree': 0,
  'Doctoral degree': 1,
  'I prefer not to answer': 2,
  'Master’s degree': 3,
  'No formal education 